In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_csv("gaku-jk2312.csv", encoding="shift-jis")
df = df.dropna()
df.head()

,四半期,国内総生産(支出側),民間最終消費支出,家計最終消費支出,除く持ち家の帰属家賃,民間住宅,民間企業設備,民間在庫変動,政府最終消費支出,公的固定資本形成,...,国内総所得,純受取,受取,支払,国民総所得,国内需要,民間需要,公的需要,総固定資本形成,最終需要
0,1994/ 1- 3.,446275.0,247502.0,243619.0,208574.0,29812.0,66259.0,4229,71357.0,47098.0,...,465653.0,4121.0,14527.0,10406.0,469774.0,462308.0,346440.0,115942.0,139766.0,442221.0
1,4- 6.,443830.0,248851.0,244912.0,209663.0,31119.0,66065.0,(2885),72244.0,48012.0,...,462572.0,3800.0,14474.0,10674.0,466372.0,460096.0,341695.0,118659.0,141500.0,445626.0
2,7- 9.,448931.0,250606.0,246645.0,211181.0,33628.0,65999.0,(281),72789.0,46294.0,...,467074.0,3871.0,14799.0,10927.0,470945.0,465711.0,348110.0,117728.0,142081.0,448128.0
3,10-12.,447124.0,250734.0,246728.0,211087.0,32218.0,67029.0,(1602),72950.0,45580.0,...,465653.0,3903.0,15316.0,11413.0,469556.0,463959.0,346832.0,117252.0,141344.0,447535.0
4,1995/ 1- 3.,452093.0,252909.0,248836.0,212992.0,31122.0,68648.0,2254,74470.0,43634.0,...,471048.0,3954.0,15604.0,11650.0,475002.0,469807.0,353586.0,116207.0,140426.0,449515.0


In [3]:
df.columns

Index(['四半期', '国内総生産(支出側)', '民間最終消費支出', '家計最終消費支出', '除く持ち家の帰属家賃', '民間住宅',
       '民間企業設備', '民間在庫変動', '政府最終消費支出', '公的固定資本形成', '公的在庫変動', '純輸出', '輸出', '輸入',
       '開差', '交易利得', '国内総所得', '純受取', '受取', '支払', '国民総所得', '国内需要', '民間需要',
       '公的需要', '総固定資本形成', '最終需要'],
      dtype='object')

In [4]:
df_ycigxm = df[["国内総生産(支出側)", 
                "民間最終消費支出",
                "民間企業設備", 
                "政府最終消費支出", 
                "輸出", 
                "輸入"
              ]]
df_ycigxm.head()

,国内総生産(支出側),民間最終消費支出,民間企業設備,政府最終消費支出,輸出,輸入
0,446275.0,247502.0,66259.0,71357.0,37756.0,45476.0
1,443830.0,248851.0,66065.0,72244.0,37964.0,46328.0
2,448931.0,250606.0,65999.0,72789.0,38658.0,47726.0
3,447124.0,250734.0,67029.0,72950.0,38821.0,48199.0
4,452093.0,252909.0,68648.0,74470.0,39312.0,49812.0


In [5]:
y = df_ycigxm["国内総生産(支出側)"]
x = df_ycigxm.drop("国内総生産(支出側)", axis=1)

In [6]:
x.corr()

,民間最終消費支出,民間企業設備,政府最終消費支出,輸出,輸入
民間最終消費支出,1.000000,0.719911,0.858213,0.907647,0.891695
民間企業設備,0.719911,1.000000,0.747032,0.816505,0.845863
政府最終消費支出,0.858213,0.747032,1.000000,0.946206,0.964487
輸出,0.907647,0.816505,0.946206,1.000000,0.976244
輸入,0.891695,0.845863,0.964487,0.976244,1.000000


In [7]:
inv = np.linalg.inv(x.corr().values)
inv

array([[  5.87459678,   0.87978464,   1.32602871,  -4.6039191 ,
         -2.76691566],
       [  0.87978464,   4.75684396,   4.72815934,  -0.22377797,
         -9.1499226 ],
       [  1.32602871,   4.72815934,  19.16353456,  -2.00658225,
        -21.70584605],
       [ -4.6039191 ,  -0.22377797,  -2.00658225,  25.09573409,
        -18.26965997],
       [ -2.76691566,  -9.1499226 , -21.70584605, -18.26965997,
         49.97746928]])

In [8]:
vif = []
for i in range(len(inv)):
    vif.append(inv[i][i])
df_vifj = pd.DataFrame(vif)
df_vifj.index = x.corr().columns
df_vifj.columns = ["VIF(実質GDP)"]
df_vifj

,VIF(実質GDP)
民間最終消費支出,5.874597
民間企業設備,4.756844
政府最終消費支出,19.163535
輸出,25.095734
輸入,49.977469


In [9]:
def partcorr(r_xy, r_yz, r_xz):
    return (r_xy - r_yz * r_xz) / (np.sqrt(1 - r_yz ** 2) * np.sqrt(1 - r_xz ** 2))

In [10]:
corr = x.corr().values
col = x.corr().columns
pcorr = []
for i in range(len(corr)):
    for j in range(i+1, len(corr[i])):
        for k in range(len(corr)):
            if col[i] != col[k] and col[j] != col[k] and col[i] and col[j]:
                pcorr.append([col[i], col[j], col[k], corr[i][j], 
                              partcorr(corr[i][j], corr[i][k], corr[k][j]), 
                              abs(corr[i][j]-partcorr(corr[i][j], corr[i][k], corr[k][j]))])
df_pcorr = pd.DataFrame(pcorr)
df_pcorr.columns = ["因子1", "因子2", "第三の因子", "相関係数", "偏相関係数", "相関係数と偏相関係数の差"]
df_pcorr.sort_values("相関係数と偏相関係数の差", ascending=False).head()

,因子1,因子2,第三の因子,相関係数,偏相関係数,相関係数と偏相関係数の差
14,民間企業設備,政府最終消費支出,輸入,0.747032,-0.488270,1.235302
17,民間企業設備,輸出,輸入,0.816505,-0.080151,0.896657
13,民間企業設備,政府最終消費支出,輸出,0.747032,-0.136773,0.883805
5,民間最終消費支出,政府最終消費支出,輸入,0.858213,-0.015185,0.873398
23,政府最終消費支出,輸出,輸入,0.946206,0.080932,0.865274


In [11]:
df_pcorr.query("因子2=='政府最終消費支出' or 因子1=='政府最終消費支出'").sort_values("相関係数と偏相関係数の差", ascending=False)

,因子1,因子2,第三の因子,相関係数,偏相関係数,相関係数と偏相関係数の差
14,民間企業設備,政府最終消費支出,輸入,0.747032,-0.488270,1.235302
13,民間企業設備,政府最終消費支出,輸出,0.747032,-0.136773,0.883805
5,民間最終消費支出,政府最終消費支出,輸入,0.858213,-0.015185,0.873398
23,政府最終消費支出,輸出,輸入,0.946206,0.080932,0.865274
4,民間最終消費支出,政府最終消費支出,輸出,0.858213,-0.004479,0.862691
12,民間企業設備,政府最終消費支出,民間最終消費支出,0.747032,0.362644,0.384388
26,政府最終消費支出,輸入,輸出,0.964487,0.581368,0.383118
21,政府最終消費支出,輸出,民間最終消費支出,0.946206,0.776301,0.169905
3,民間最終消費支出,政府最終消費支出,民間企業設備,0.858213,0.694432,0.163780
24,政府最終消費支出,輸入,民間最終消費支出,0.964487,0.857477,0.107010


In [12]:
x = sm.add_constant(x).dropna()
model = model = sm.OLS(y, x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             国内総生産(支出側)   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     2601.
Date:                Fri, 07 Jul 2023   Prob (F-statistic):          3.01e-113
Time:                        03:05:14   Log-Likelihood:                -1095.0
No. Observations:                 117   AIC:                             2202.
Df Residuals:                     111   BIC:                             2219.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.633e+05   1.31e+04     12.453      0.000    1.37e+05    1.89e+05
民間最終消費支出       0.6551      0.041     16.031      0.000       0.574       0.736
民間企業設備         1.3639      0.086     15.901      0.000       1.194       1.534
政府最終消費支出       0.2170      0.091      2.374      0.019       0.036       0.398
輸出             0.5887      0.059      9.952      0.000       0.472       0.706
輸入            -0.1796      0.106     -1.692      0.093      -0.390       0.031
==============================================================================
Omnibus:                       20.060   Durbin-Watson:                   0.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.245
Skew:                          -0.824   Prob(JB):                     2.71e-07
Kurtosis:                       4.868   Cond. No.                     1.62e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.62e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
x

,const,民間最終消費支出,民間企業設備,政府最終消費支出,輸出,輸入
0,1.0,247502.0,66259.0,71357.0,37756.0,45476.0
1,1.0,248851.0,66065.0,72244.0,37964.0,46328.0
2,1.0,250606.0,65999.0,72789.0,38658.0,47726.0
3,1.0,250734.0,67029.0,72950.0,38821.0,48199.0
4,1.0,252909.0,68648.0,74470.0,39312.0,49812.0
...,...,...,...,...,...,...
112,1.0,289846.0,86923.0,118148.0,105025.0,105988.0
113,1.0,294730.0,88799.0,118945.0,106601.0,107002.0
114,1.0,294919.0,90149.0,118976.0,109221.0,112996.0
115,1.0,295556.0,89625.0,119189.0,111447.0,112949.0


In [14]:
df = pd.read_csv("gaku-mk2312.csv", encoding="shift-jis")
df = df.dropna()
df.head()

,四半期,国内総生産(支出側),民間最終消費支出,家計最終消費支出,除く持ち家の帰属家賃,民間住宅,民間企業設備,民間在庫変動,政府最終消費支出,公的固定資本形成,...,輸入,純受取,受取,支払,国民総所得,国内需要,民間需要,公的需要,総固定資本形成,最終需要
0,1994/ 1- 3.,510129.0,266613.0,262049.0,227606.0,28674.0,80433.0,3822,73547.0,46403.0,...,33933.0,4449.0,15710.0,11261.0,514578.0,499075.0,379542.0,119533.0,155511.0,506725.0
1,4- 6.,509013.0,269151.0,264515.0,229700.0,29970.0,79980.0,(3045),75113.0,47380.0,...,35524.0,4148.0,15650.0,11502.0,513161.0,499037.0,376056.0,122982.0,157330.0,511570.0
2,7- 9.,512969.0,270651.0,265985.0,230762.0,32363.0,79431.0,(138),75898.0,45541.0,...,36487.0,4190.0,16018.0,11828.0,517159.0,504402.0,382306.0,122096.0,157334.0,512450.0
3,10-12.,511573.0,271237.0,266518.0,230814.0,30940.0,80299.0,(1605),76697.0,44714.0,...,36525.0,4208.0,16563.0,12355.0,515781.0,502988.0,380871.0,122117.0,155953.0,512471.0
4,1995/ 1- 3.,514942.0,272765.0,267977.0,231905.0,29884.0,82050.0,2065,77679.0,42840.0,...,37246.0,4262.0,16862.0,12600.0,519205.0,507135.0,386764.0,120371.0,154774.0,513025.0


In [15]:
df.columns

Index(['四半期', '国内総生産(支出側)', '民間最終消費支出', '家計最終消費支出', '除く持ち家の帰属家賃', '民間住宅',
       '民間企業設備', '民間在庫変動', '政府最終消費支出', '公的固定資本形成', '公的在庫変動', '純輸出', '輸出', '輸入',
       '純受取', '受取', '支払', '国民総所得', '国内需要', '民間需要', '公的需要', '総固定資本形成', '最終需要'],
      dtype='object')

In [16]:
df_ycigxm = df[["国内総生産(支出側)", 
                "民間最終消費支出",
                "民間企業設備", 
                "政府最終消費支出", 
                "輸出", 
                "輸入"
              ]]
df_ycigxm.head()

,国内総生産(支出側),民間最終消費支出,民間企業設備,政府最終消費支出,輸出,輸入
0,510129.0,266613.0,80433.0,73547.0,44988.0,33933.0
1,509013.0,269151.0,79980.0,75113.0,45500.0,35524.0
2,512969.0,270651.0,79431.0,75898.0,45054.0,36487.0
3,511573.0,271237.0,80299.0,76697.0,45110.0,36525.0
4,514942.0,272765.0,82050.0,77679.0,45054.0,37246.0


In [17]:
y = df_ycigxm["国内総生産(支出側)"]
x = df_ycigxm.drop("国内総生産(支出側)", axis=1)

In [18]:
x.corr()

,民間最終消費支出,民間企業設備,政府最終消費支出,輸出,輸入
民間最終消費支出,1.000000,0.512830,0.810333,0.879074,0.855870
民間企業設備,0.512830,1.000000,0.283093,0.448139,0.401654
政府最終消費支出,0.810333,0.283093,1.000000,0.886844,0.898936
輸出,0.879074,0.448139,0.886844,1.000000,0.964391
輸入,0.855870,0.401654,0.898936,0.964391,1.000000


In [19]:
inv = np.linalg.inv(x.corr().values)
inv

array([[  5.05498771,  -0.92959844,  -1.17242164,  -2.73752796,
         -0.25905898],
       [ -0.92959844,   1.52760611,   0.90964133,  -0.8731547 ,
          0.20639961],
       [ -1.17242164,   0.90964133,   5.99634811,  -1.59823282,
         -3.21093452],
       [ -2.73752796,  -0.8731547 ,  -1.59823282,  17.61046056,
        -12.85297904],
       [ -0.25905898,   0.20639961,  -3.21093452, -12.85297904,
         16.42053866]])

In [20]:
vif = []
for i in range(len(inv)):
    vif.append(inv[i][i])
df_vif = pd.DataFrame(vif)
df_vif.columns = ["VIF(名目GDP)"]
df_vif.index = x.corr().columns
df_vif

,VIF(名目GDP)
民間最終消費支出,5.054988
民間企業設備,1.527606
政府最終消費支出,5.996348
輸出,17.610461
輸入,16.420539


In [21]:
corr = x.corr().values
col = x.corr().columns
pcorr = []
for i in range(len(corr)):
    for j in range(i+1, len(corr[i])):
        for k in range(len(corr)):
            if col[i] != col[k] and col[j] != col[k] and col[i] and col[j]:
                pcorr.append([col[i], col[j], col[k], corr[i][j], 
                              partcorr(corr[i][j], corr[i][k], corr[k][j]), 
                              abs(corr[i][j]-partcorr(corr[i][j], corr[i][k], corr[k][j]))])
df_pcorr = pd.DataFrame(pcorr)
df_pcorr.columns = ["因子1", "因子2", "第三の因子", "相関係数", "偏相関係数", "相関係数と偏相関係数の差"]
df_pcorr.sort_values("相関係数と偏相関係数の差", ascending=False).head()

,因子1,因子2,第三の因子,相関係数,偏相関係数,相関係数と偏相関係数の差
11,民間最終消費支出,輸入,輸出,0.855870,0.064247,0.791623
23,政府最終消費支出,輸出,輸入,0.886844,0.171912,0.714932
4,民間最終消費支出,政府最終消費支出,輸出,0.810333,0.139523,0.670810
5,民間最終消費支出,政府最終消費支出,輸入,0.810333,0.180782,0.629551
13,民間企業設備,政府最終消費支出,輸出,0.283093,-0.276795,0.559888


In [22]:
df_pcorr.query("因子2=='政府最終消費支出' or 因子1=='政府最終消費支出' or 第三の因子=='政府最終消費支出'").sort_values("相関係数と偏相関係数の差", ascending=False)

,因子1,因子2,第三の因子,相関係数,偏相関係数,相関係数と偏相関係数の差
23,政府最終消費支出,輸出,輸入,0.886844,0.171912,0.714932
4,民間最終消費支出,政府最終消費支出,輸出,0.810333,0.139523,0.670810
5,民間最終消費支出,政府最終消費支出,輸入,0.810333,0.180782,0.629551
13,民間企業設備,政府最終消費支出,輸出,0.283093,-0.276795,0.559888
12,民間企業設備,政府最終消費支出,民間最終消費支出,0.283093,-0.263333,0.546427
26,政府最終消費支出,輸入,輸出,0.898936,0.357356,0.541580
14,民間企業設備,政府最終消費支出,輸入,0.283093,-0.194342,0.477435
10,民間最終消費支出,輸入,政府最終消費支出,0.855870,0.496424,0.359446
7,民間最終消費支出,輸出,政府最終消費支出,0.879074,0.592538,0.286535
21,政府最終消費支出,輸出,民間最終消費支出,0.886844,0.624730,0.262114


In [23]:
x = sm.add_constant(x)
model = model = sm.OLS(y, x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             国内総生産(支出側)   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     407.9
Date:                Fri, 07 Jul 2023   Prob (F-statistic):           1.08e-69
Time:                        03:05:14   Log-Likelihood:                -1140.3
No. Observations:                 117   AIC:                             2293.
Df Residuals:                     111   BIC:                             2309.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.092e+05   2.36e+04      4.619      0.000    6.24e+04    1.56e+05
民間最終消費支出       0.5782      0.096      6.043      0.000       0.389       0.768
民間企業設備         2.4167      0.077     31.196      0.000       2.263       2.570
政府最終消費支出       0.7094      0.084      8.488      0.000       0.544       0.875
輸出             0.4726      0.081      5.803      0.000       0.311       0.634
輸入            -0.7608      0.062    -12.329      0.000      -0.883      -0.639
==============================================================================
Omnibus:                       11.091   Durbin-Watson:                   0.760
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               11.647
Skew:                          -0.651   Prob(JB):                      0.00296
Kurtosis:                       3.834   Cond. No.                     2.03e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.03e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""